In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#importing necessary libararies
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import itertools
import collections
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

In [3]:
#reading csv file
corpus=pd.read_csv("/content/drive/MyDrive/DSF trials/preprocessed_annotated_data.csv")

### Text cleaning

Stopword removal

In [6]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

# clean dataset
def clean_dataset(text):
    text = text.lower()
    text = re.sub(r'#','', text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    text = re.sub(r'([\w\d]+)([^\w\d ]+)', '\1 \2',text)
    text = re.sub(r'([^\w\d ]+)([\w\d]+)', '\1 \2',text)
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = ' '.join(re.sub("[\.\,\!\?\:\;\-\=\/\|\'\(\']", " ", text).split())
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

#remove stop word i.e. the most frequently appeared words 
myOwnStopList=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'what','how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'do', 'should', "should", 'now', 'd', 'm', 'o', 're', 've', 'y', 'ain', "are", 'could', "was",
'would','have','get','got','getting','one','two','still','going']


In [7]:
corpus['clean_plot_summary']=corpus['clean_plot_summary'].apply(lambda x: remove_contractions(str(x)))

corpus['clean_plot_summary'] =corpus['clean_plot_summary'].apply(lambda x: clean_dataset(x))
corpus['clean_plot_summary'] = corpus['clean_plot_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (myOwnStopList)]))

Tuning the output for calling multilabel classification

In [9]:
import ast
corpus['multi_label_genre']=corpus['multi_label_genre'].apply(lambda x: ast.literal_eval(x))

### Multilabel classification output transformation

Calling in Multilabelbinarizer

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()

# transform target variable
y = multilabel_binarizer.fit_transform(corpus['multi_label_genre'])

In [11]:
print(y.shape)

(13132, 30)


In [12]:
x=corpus['clean_plot_summary']

In [13]:
print(x.shape)

(13132,)


### Train Test split

In [14]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

### Feature transformation

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=15000,stop_words='english')


In [16]:
xtrain = vect.fit_transform(xtrain)
xtest=vect.transform(xtest)

In [17]:
print(xtrain.shape)
print(xtest.shape)

(10505, 15000)
(2627, 15000)


Binary Relevance 

In [18]:
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)
from sklearn.ensemble import RandomForestClassifier
model_br=RandomForestClassifier()
model_cc=RandomForestClassifier()



In [19]:
predictions = pd.DataFrame()

In [21]:
from tqdm import tqdm
for i in tqdm(range(y.shape[1])):
    ytrain_i = ytrain[:,i]
    label="genre"+str(i)
    
    # train the model using xtrain & this particular genre(y[:,i])
    model_br.fit(xtrain, ytrain_i)
    
    # compute the training accuracy
    ypred_i = model_br.predict(xtest)
    
    # compute the predicted probabilities for xtest
    test_y_prob = model_br.predict_proba(xtest)[:,0]
    predictions[label] = 1.0-test_y_prob

100%|██████████| 30/30 [07:40<00:00, 15.34s/it]


Classifier Chains

In [22]:
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [23]:
predictions2 = pd.DataFrame()

In [25]:
for i in tqdm(range(y.shape[1])):
    ytrain_i = ytrain[:,i]
    label="genre"+str(i)
    
    # train the model using xtrain & this particular genre(y[:,i])
    model_cc.fit(xtrain, ytrain_i)
    
    # compute the training accuracy
    y_pred_train_i=model_cc.predict(xtrain)
    ypred_i = model_cc.predict(xtest)
    
    # compute the predicted probabilities for xtest
    test_y_prob = model_cc.predict_proba(xtest)[:,0]
    predictions2[label] = 1.0-test_y_prob
    add_feature(xtrain,y_pred_train_i)
    add_feature(xtest,ypred_i)
    

100%|██████████| 30/30 [07:50<00:00, 15.68s/it]


Merging results of Binary Relevance and Classifier Chaining

In [26]:
final_y_test_pred=predictions+predictions2

In [27]:
res=final_y_test_pred.to_numpy()
res1=final_y_test_pred.to_numpy()

In [28]:
ytest[1]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
res[1]

array([0.36, 0.41, 0.53, 0.04, 0.01, 0.15, 0.04, 0.04, 0.02, 0.12, 0.  ,
       0.04, 0.03, 0.03, 0.4 , 0.73, 0.05, 0.  , 0.96, 0.  , 0.51, 0.24,
       0.03, 0.01, 0.05, 0.01, 0.03, 0.46, 0.39, 0.13])

### Converting logits into binary probabilities

In [30]:
for i in range(res.shape[0]):
  for j in range(res.shape[1]):
    if res[i][j]>0.5:
      res[i][j]=1
    else:
      res[i][j]=0

### Inference

In [32]:
for i in range(0,50):
  print("\n trial {} ".format(i))
  print("Actual: ",multilabel_binarizer.inverse_transform(ytest)[i])
  print("Predicted: ",multilabel_binarizer.inverse_transform(res)[i])




 trial 0 
Actual:  ('fiction', 'literature', 'nonfiction')
Predicted:  ('children', 'fiction', 'literature', 'nonfiction', 'romance')

 trial 1 
Actual:  ('adventure', 'fiction', 'literature')
Predicted:  ('anthropology', 'fiction', 'literature', 'mystery')

 trial 2 
Actual:  ('biography', 'business', 'fantasy')
Predicted:  ('adventure', 'biography', 'history', 'literature', 'nonfiction')

 trial 3 
Actual:  ('fiction', 'literature', 'nonfiction')
Predicted:  ('adventure', 'fiction', 'literature', 'nonfiction', 'romance')

 trial 4 
Actual:  ('environmental', 'history', 'utopian')
Predicted:  ('adventure', 'documentary', 'fiction', 'literature')

 trial 5 
Actual:  ('fiction', 'literature', 'nonfiction')
Predicted:  ('fiction', 'literature', 'nonfiction')

 trial 6 
Actual:  ('fiction', 'nonfiction', 'thriller')
Predicted:  ('adventure', 'fiction', 'literature', 'nonfiction', 'thriller')

 trial 7 
Actual:  ('fiction', 'literature', 'utopian')
Predicted:  ('action', 'fiction', 'liter

In [33]:
def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

def Precision(y_true, y_pred):
      temp = 0
      for i in range(y_true.shape[0]):
          if sum(y_true[i]) == 0:
              continue
          temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
      return temp/ y_true.shape[0]

def Recall(y_true, y_pred):
      temp = 0
      for i in range(y_true.shape[0]):
          if sum(y_pred[i]) == 0:
              continue
          temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
      return temp/ y_true.shape[0]

def F1Measure(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
    return temp/ y_true.shape[0]
    
    print(F1Measure(y_true, y_pred))
    
def Hamming_Loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])



In [34]:
print("\n Accuracy: ",Accuracy(ytest,res1))
print("\n Precision: ",Precision(ytest,res1))
print("\n Recall: ",Recall(ytest,res1))
print("\n F1 Measure: ",F1Measure(ytest,res1))
print("\n Hamming Loss: : ",Hamming_Loss(ytest,res1))


 Accuracy:  0.3609825757713091

 Precision:  0.52328384722751

 Recall:  0.4739849278581678

 F1 Measure:  0.47859543282078537

 Hamming Loss: :  0.10721989595229031


Saving learned models

In [35]:
import pickle
filename = '/content/drive/MyDrive/DSF trials/classifier_models/tf_idf_binary_relevance_model.sav'
pickle.dump(model_br, open(filename, 'wb'))

filename = '/content/drive/MyDrive/DSF trials/classifier_models/tf_idf_classifier_chain_model.sav'
pickle.dump(model_cc, open(filename, 'wb'))